# Get GDELT Schema

In [67]:
import urllib.request  # lib that handles URLs
#import requests
import io
import pandas as pd
pd.options.display.max_rows=250

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data))
colnames_events = list(formats["tableId"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_mentions = list(formats.loc[:15,"0"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_knowledgegraph = list(formats["tableId"])

In [73]:
len(colnames_knowledgegraph)

27

# Spark Dataset Import

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [ ]:
df_events = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/events/20180101*.export.csv")
df_events = df_events.toDF(*colnames_events)
df_events.cache()

In [ ]:
df_mentions = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/mentions/20180101*.mentions.csv")
df_mentions = df_mentions.toDF(*colnames_mentions)
df_mentions.cache()

In [ ]:
df_gkg = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/gkg/20180101*.gkg.csv")
df_gkg = df_gkg.toDF(*colnames_knowledgegraph)
df_gkg.cache()

In [75]:
df_gkg.count()

130296

In [ ]:
df_gkg.printSchema()

# Basic Queries

In [78]:
recs = pd.DataFrame(df_gkg.take(3))
recs.columns = colnames_knowledgegraph
recs.T

,0,1,2
GKGRECORDID,20180101193000-0,20180101193000-1,20180101193000-2
DATE,20180101193000,20180101193000,20180101193000
SourceCollectionIdentifier,1,1,1
SourceCommonName,ndtv.com,news.statetimes.in,egmnow.com
DocumentIdentifier,https://www.ndtv.com/india-news/us-president-d...,http://news.statetimes.in/81-78-lakh-pilgrims-...,http://www.egmnow.com/articles/previews/soulca...
Counts,None,None,None
V2Counts,None,None,None
Themes,LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDENT;USPEC_P...,LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;TAX_FNCA...,None
V2Themes,"TAX_ETHNICITY_INDIAN,397;MILITARY,402;TAX_FNCA...","ECON_STOCKMARKET,370;ECON_STOCKMARKET,1482;TAX...",None
Locations,1#Afghanistan#AF#AF#33#66#AF;1#United States#U...,"4#Jammu, Jammu And Kashmir, India#IN#IN12#32.7...",1#Japan#JA#JA#36#138#JA


# Stop Spark Context

In [ ]:
sc.stop()